# Data Exploration

The finance team wants to forecast sales in all their stores across several cities six weeks ahead of time.

- Id - an Id that represents a (Store, Date) duple within the test set
- Store - a unique Id for each store
- Sales - the turnover for any given day (this is what you are predicting)
- Customers - the number of customers on a given day
- Open - an indicator for whether the store was open: 0 = closed, 1 = open
- StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
- SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools
- StoreType - differentiates between 4 different store models: a, b, c, d
- Assortment - describes an assortment level: a = basic, b = extra, c = extended. Read more about assortment here
- CompetitionDistance - distance in meters to the nearest competitor store
- CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened
- Promo - indicates whether a store is running a promo on that day
- Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
- Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2
- PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

## Loading Data

In [1]:
#importing Libraries
import pandas as pd
# import dvc.api
import os
import sys

In [2]:
#import local libraries
#Adding scripts path
sys.path.append(os.path.abspath(os.path.join('..')))
#importing dvc_data_loader script
# from scripts.dvc_data_loader import *
from scripts.data_information import DataInfo
from scripts.data_loader import load_df_from_csv
from scripts.data_manipulation import DataManipulator
from scripts.data_cleaner import DataCleaner
from scripts.utlity_functions import convert_to_month_name
from scripts.grapher import *


## Check for seasonality in both training and test sets - are the seasons similar between these two groups?

In [3]:
#Loading the train data
train_df = load_df_from_csv('../data/train.csv')
train_data_info = DataInfo(train_df, deep=True)
train_data_info.get_information()
train_data_info.get_col_unique_value_count('Store')

C:\Python38\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


DataFrame Information: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Store          1017209 non-null  uint16
 1   DayOfWeek      1017209 non-null  uint8 
 2   Date           1017209 non-null  object
 3   Sales          1017209 non-null  uint16
 4   Customers      1017209 non-null  uint16
 5   Open           1017209 non-null  uint8 
 6   Promo          1017209 non-null  uint8 
 7   StateHoliday   1017209 non-null  object
 8   SchoolHoliday  1017209 non-null  uint8 
dtypes: object(2), uint16(3), uint8(4)
memory usage: 25.2+ MB
Number of unique values in column Store is: 1115


In [4]:
#Loading the store data
store_df = load_df_from_csv('../data/store.csv')
store_data_info = DataInfo(store_df, deep=True)
store_data_info.get_information()


DataFrame Information: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store                      1115 non-null   uint16 
 1   StoreType                  1115 non-null   object 
 2   Assortment                 1115 non-null   object 
 3   CompetitionDistance        1112 non-null   float32
 4   CompetitionOpenSinceMonth  761 non-null    float32
 5   CompetitionOpenSinceYear   761 non-null    float32
 6   Promo2                     1115 non-null   uint8  
 7   Promo2SinceWeek            571 non-null    float32
 8   Promo2SinceYear            571 non-null    float32
 9   PromoInterval              571 non-null    object 
dtypes: float32(5), object(3), uint16(1), uint8(1)
memory usage: 51.3+ KB


In [5]:
# Combining Train Data with Store Data using Store Id to match/join the instances
joined_df = DataCleaner(train_df)
joined_df = joined_df.add_columns_from_another_df_using_column(store_df, 'Store', ['StoreType','Assortment','CompetitionDistance','Promo2'])
joined_df

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,0
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,1
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,0
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1,a,a,1900.0,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1,c,c,1880.0,0
1017206,1113,2,2013-01-01,0,0,0,0,a,1,a,c,9260.0,0
1017207,1114,2,2013-01-01,0,0,0,0,a,1,a,c,870.0,0


In [7]:
# joined_df.to_csv('../data/store_train_joined.csv')

In [16]:
joined_df['StateHoliday'] = joined_df['StateHoliday'].apply(lambda x: 0 if x == '0' else x)


In [ ]:
#Loading the joined data
joined_df = load_df_from_csv('../data/store_train_joined.csv')
joined_df_info = DataInfo(joined_df, deep=True)
joined_df_info.get_information()


In [ ]:
#Loading the test data
test_df = load_df_from_csv('../data/test.csv')
test_data_info = DataInfo(test_df, deep=True)
test_data_info.get_information()


In [ ]:
# We have to separate the Date to determine season
# Separating date for train_data
train_data_cleaner = DataCleaner(train_df)
test_data_cleaner = DataCleaner(test_df)

In [ ]:
train_data_cleaner.change_column_to_date_type('Date')
test_data_cleaner.change_column_to_date_type('Date')

In [ ]:
test_data_cleaner.df.dtypes

In [ ]:
train_data_cleaner.separate_date_column(date_column='Date')
test_data_cleaner.separate_date_column(date_column='Date')

In [ ]:
train_season = DataInfo(train_data_cleaner.df)
test_season = DataInfo(test_data_cleaner.df)

In [ ]:
train_season.get_col_unique_value_count('Month')

In [ ]:
train_season.get_col_value_count('Month')

In [ ]:
test_season.get_col_unique_value_count('Month')

In [ ]:
test_season.get_col_value_count('Month')

In [ ]:
#Checking if they seasons are the same(month)
train_season.get_col_value_count('Month').index.to_list() == test_season.get_col_value_count('Month').index.to_list()

In [ ]:
plt.title('A tale of 2 subplots')

plt.subplot(1, 2, 1)
a = train_season.df['Month'].value_counts()
plt.bar(a.index, a.values)
plt.ylabel('Damped oscillation')


plt.subplot(1, 2, 2)
train_season.df['Month'].value_counts().plot(kind='bar')
plt.xlabel('time (s)')
plt.ylabel('Undamped')

fig.tight_layout()


In [ ]:
# graph_comparison_bar_plot(train_season.df,test_season.df, 'Month', "Season")
import seaborn as sns
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
sns.histplot(train_season.df['Month'], ax=ax1)
sns.histplot(test_season.df['Month'], ax=ax2)
fig.suptitle(f'Distribution of hello', size=20, fontweight='bold')
fig.tight_layout()


## Check & compare sales behavior before, during and after holidays

## Find out any seasonal (Christmas, Easter etc) purchase behaviours

In [ ]:
train_season.df['StateHoliday'] = train_season.df['StateHoliday'].apply(lambda x: if(x == 0 or x == '0') return 0)
seasonal = train_season.get_grouped_by('StateHoliday').agg({'Sales':'mean', 'Customers':'mean'})
# seasonal = seasonal.iloc[2:,:]
seasonal

In [ ]:
import numpy as np

seasonal.index = ['Public Holiday','Easter Holiday','Christmas']
X_axis = np.arange(len(seasonal.index))
plt.bar(X_axis - 0.2, seasonal['Sales'], 0.4, label='Sales')
plt.bar(X_axis + 0.2, seasonal['Customers'], 0.4, label='Customers')

plt.xticks(X_axis, seasonal.index)
plt.xlabel("Season")
plt.ylabel("Amount")
plt.title("Sales and Customers During Holiday Seasons")
plt.legend()
plt.show()


## What can you say about the correlation between sales and number of customers?

In [ ]:
sales_customer_rln = train_df.loc[:,['Sales','Customers']]
sales_cutomer_corr = sales_customer_rln.corr()
sales_cutomer_corr

In [ ]:
plt.figure(figsize=(10, 7))
sns.heatmap(sales_cutomer_corr)
plt.title('Sales - Customer Relation ', size=18, fontweight='bold')

## How does promo affect sales? Are the promos attracting more customers? How does it affect already existing customers?

### How does promo affect sales?

In [ ]:
sales_promo_rln = train_df.loc[:,['Sales','Promo']]
sales_promo_corr = sales_promo_rln.corr()
sales_promo_corr

In [ ]:
plt.figure(figsize=(10, 7))
sns.heatmap(sales_promo_corr)
plt.title('Sales - Promotion Relation', size=18, fontweight='bold')


### Are the promos attracting more customers?

In [ ]:
customer_promo_rln = train_df.loc[:, ['Customers', 'Promo']]
customer_promo_corr = customer_promo_rln.corr()
customer_promo_corr


In [ ]:
plt.figure(figsize=(10, 7))
sns.heatmap(customer_promo_corr)
plt.title('Sales - Promotion Relation', size=18, fontweight='bold')


In [ ]:
promo_customers_rln = train_df.loc[:,['Store','Customers','Promo']]
store_customer_difference = promo_customers_rln.groupby(['Store','Promo']).agg({'Customers':'mean'})
store_customer_difference

## Trends of customer behavior during store open and closing times

In [ ]:
customer_value = store_customer_difference.Customers.values.tolist()
def get_average_increase_from_all_stores(grouped_cutomer_list:list) -> float:
    percentage_increase = []
    i = 0
    while(i < len(grouped_cutomer_list)):
        new_perc = ((grouped_cutomer_list[i+1] * 100) / grouped_cutomer_list[i]) - 100
        percentage_increase.append(new_perc)
        i += 2

    return sum(percentage_increase) / len(percentage_increase)

get_average_increase_from_all_stores(customer_value)

## Could the promos be deployed in more effective ways? Which stores should promos be deployed in?

In [ ]:
# Promos are better deployed if they show a high percentage of increase
def get_increase_per_store(grouped_cutomer_list: list, profitable_promo:float=10) -> list:
    store_index = []
    i = 0
    while(i < len(grouped_cutomer_list)):
        new_perc = ((grouped_cutomer_list[i+1] * 100) / grouped_cutomer_list[i]) - 100
        if(new_perc >= profitable_promo):
            store_index.append(i/2)
        i += 2

    return store_index


store_index = get_increase_per_store(customer_value, 10)
store_customer_difference.iloc[store_index,:]


## Trends of customer behavior during store open and closing times

In [ ]:
customer_behaviour = train_df.groupby('Open').agg({'Customers':'mean','Sales':'mean'})
customer_behaviour

## Which stores are opened on all weekdays? How does that affect their sales on weekends? 

In [ ]:
# open_all_weekdays = 
# all_week_stores = pd.DataFrame(train_df.groupby(['Store', 'DayOfWeek']).size().groupby(level=1).count(),columns=['Open For Week Days'])
# all_week_stores[all_week_stores['Open For Week Days'] == 5]
# all_week_stores
# train_df.groupby(['Store', 'DayOfWeek']).agg({'Sales':'mean'})


In [36]:
# open_all_weekdays (Mon - Fri)
# First Remove all holiday related days
holiday_out_df = joined_df[joined_df['StateHoliday'] == 0]
weekdays_df = holiday_out_df[holiday_out_df['DayOfWeek'] <= 5]
not_open_stores = weekdays_df[weekdays_df['Open'] == 0]
not_open_stores_id = not_open_stores['Store']
not_open_stores_id = not_open_stores_id.values.tolist()

all_weekdays_open_stores = joined_df[~joined_df['Store'].isin(not_open_stores_id)]
all_weekdays_open_stores


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,0
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,0
5,6,5,2015-07-31,5651,589,1,1,0,1,a,a,310.0,0
6,7,5,2015-07-31,15344,1414,1,1,0,1,a,c,24000.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1,a,a,1900.0,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1,c,c,1880.0,0
1017206,1113,2,2013-01-01,0,0,0,0,a,1,a,c,9260.0,0
1017207,1114,2,2013-01-01,0,0,0,0,a,1,a,c,870.0,0


## Check how the assortment type affects sales

In [ ]:
assortment_sales_rln = train_df.loc[:, ['Assortment', 'Sales']]
assortment_sales_rln = assortment_sales_rln.groupby(
    "Assortment").agg({'Sales': 'sum'})
assortment_sales_rln
# plt.bar(X_axis - 0.2, seasonal['Sales'], 0.4, label='Sales')


In [ ]:
# Adding Season Column to both train and test dataframes
# Train DataFrame
# Creating a DataCleaner Class to access the add season method
train_add_season = DataCleaner(train_season.df)
# Using the add season method
train_add_season.add_season_col('Month')

# Creating a DataCleaner Class to access the add season method
test_add_season = DataCleaner(test_season.df)
# Using the add season method
test_add_season.add_season_col('Month')


In [ ]:
# Creating a DataInfo class on both to use information methods
train_season = DataInfo(train_add_season.df)
test_season = DataInfo(test_add_season.df)

In [ ]:
# sample of train season after getting new season column
train_season.df.sample(10)

In [ ]:
# Grouping train data by Month and summing the sales and customers columns
month_train = train_season.get_grouped_by('Month').agg({'Sales':'sum','Customers':'sum'}).sort_values(by='Sales',ascending=False)
month_train

In [ ]:
# Sales where high in this months respectively
convert_to_month_name(month_train.index.to_list())

In [ ]:
# Grouping train data by Season and summing the sales and customers columns
train_season.get_grouped_by('Season').agg(
    {'Sales': 'sum', 'Customers': 'sum'}).sort_values(by='Sales', ascending=False)


In [ ]:
test_season.get_grouped_by('Month').agg(
    {'Sales': 'sum', 'Customers': 'sum'}).sort_values(by='Sales', ascending=False)


In [ ]:
test_season.get_grouped_by('Season').agg(
    {'Sales': 'sum', 'Customers': 'sum'}).sort_values(by='Sales', ascending=False)

In [ ]:
train_data_info.df.head(5)

In [ ]:
test_data_cleaner.df.info()

In [ ]:
train_data_info.get_size()

In [ ]:
#Check if there are columns with missing values
train_data_info.get_columns_with_missing_values()

In [ ]:
train_data_info.get_columns()

In [ ]:
train_data_info.get_dispersion_params()

In [ ]:
train_data_info.get_duplicates()

In [ ]:
train_data_info.get_matrix_correlation()

In [ ]:
train_data_info.get_min_max_of_dataframe_columns()

In [ ]:
store_group = train_data_info.get_grouped_by('Store').agg({'Sales':'sum','Customers':'sum','Open':'sum', 'Promo':'sum','SchoolHoliday':'sum'})
store_group

In [ ]:
store_group.corr()